In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.preprocessing import MultiLabelBinarizer


In [ ]:

# ✅ CONFIG
N_CLASSES = 3  # pulse, broad, narrow
SLICE_HEIGHT = 1  # 1 time-step ต่อ slice
IMG_HEIGHT = 256
IMG_WIDTH = 256
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Map label name → index
LABEL_MAP = {'pulse': 0, 'broad': 1, 'narrow': 2}

# ✅ Dataset
class FRBDataset(Dataset):
    def __init__(self, npy_path, label_csv, transform=None):
        self.data = np.load(npy_path)
        self.labels = pd.read_csv(label_csv)
        self.transform = transform
        self.samples = []
        for _, row in self.labels.iterrows():
            idx = int(row['index'])
            raw_label = row['labels'].lower().split('+')
            label = [0] * N_CLASSES
            for l in raw_label:
                if l in LABEL_MAP:
                    label[LABEL_MAP[l]] = 1
            self.samples.append((idx, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, i):
        idx, label = self.samples[i]
        img = self.data[idx:idx+1, :]  # slice: 1 × 256
        img = np.repeat(img, 3, axis=0)  # → 3 × 256
        img = torch.tensor(img, dtype=torch.float32)
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.float32)

# ✅ Model (Light CNN)
class SimpleCNN(nn.Module):
    def __init__(self, out_dim=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(16, out_dim)
        )

    def forward(self, x):
        return self.net(x)

# ✅ Load Data
dataset = FRBDataset(
    npy_path="yourfile.npy",
    label_csv="yourfile_labels.csv",
    transform=None
)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# ✅ Train
model = SimpleCNN(out_dim=N_CLASSES).to(DEVICE)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(5):
    model.train()
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        out = model(x.unsqueeze(1))  # Add channel dim
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} Loss: {loss.item():.4f}")

# ✅ Predict (example)
model.eval()
test_data = np.load("yourfile.npy")
results = []
for i in range(test_data.shape[0]):
    x = test_data[i:i+1, :]
    x = torch.tensor(x, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(DEVICE)
    pred = torch.sigmoid(model(x)).detach().cpu().numpy()[0]
    pred_bin = (pred > 0.5).astype(int)
    results.append([f"0_{i}"] + pred_bin.tolist())

# ✅ Save submission
submission_df = pd.DataFrame(results, columns=["id", "pulse", "broad", "narrow"])
submission_df.to_csv("submission.csv", index=False)
print("✅ Done: submission.csv saved")
